# CSCI 3022 Homework
<figure>
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right">
</figure>

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below. 

In [2]:
NAME = "Tyler Cranmer"
COLLABORATORS = ""

If you referenced any web sites or solutions not of your own creation, list those references here:

* List any external references or resources here

---

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import stats
%matplotlib inline
%matplotlib inline
import numpy as np
import scipy as sp
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import random
import math

# Overview

In this homework, you're going to compute confidence intervals for two different datasets using multiple methods. The first data set is quite large, and you should find that the different techniques yield similar answers. The second dataset is much smaller. You will then solve a problem concerning confidence intervals for proportions (*e.g.* polling results).

# Problem #1 - Traffic Comparison [50 pts]

The British [Luddite](https://en.wikipedia.org/wiki/Luddite) society argues that modern traffic lights cause more accidents than traditional "give way" (yield) intersections and that the problem is getting worse over time.

Being a smart data scientist, you realize that "correlation can't prove causation", but you can at least determine if automatic intersections have more dangerous accidents that traditional intersections. Thinking quickly you [realize that traffic accident data is available](https://www.kaggle.com/daveianhickey/2000-16-traffic-flow-england-scotland-wales) and has been provided to you for the 2005-2007 year spand in the file 'accidents_2005_to_2007.csv'.

In [4]:
df = pd.read_csv('accidents_2005_to_2007.csv', dtype='unicode')

This datafile contains a number of columns [as described here](https://www.kaggle.com/daveianhickey/2000-16-traffic-flow-england-scotland-wales/data). The data has a number of columns of interest. However, becasue the data set is large and somewhat unclean, you'll need to convert columns of interest into the appropriate data types. You should convert the following columns to their appropriate data types:
* Number_of_Casualties
* Junction_Control
* Year

Modify the dataframe `df` to have these columns converted to the appropriate types.

In [5]:
# your code here
df = pd.DataFrame(df)
df['Number_of_Casualties'] = df['Number_of_Casualties'].astype('int64')
df['Junction_Control'] = df['Junction_Control'].astype('str')
df['Year'] = df['Year'].astype('int64')


The 'Junction_Control' has a single machine-controlled traffic mechanism, 'Automatic traffic signal'. The other methods are all human controlled. Create a new column labeled `machine` that distinguishes the accidents involving automatic traffic light controls from the other accidents (e.g. stop sign, yield, etc).

In [6]:
# your code here
list = []
for row in df['Junction_Control']:
        list.append(row)


df['machine'] = list
df['machine'] = df['machine'].replace(['Stop Sign','Giveway or uncontrolled', 'Authorised person', 'nan'],'other')
df['machine'] 

0                            other
1         Automatic traffic signal
2                            other
3                            other
4                            other
                    ...           
570006                       other
570007                       other
570008                       other
570009                       other
570010                       other
Name: machine, Length: 570011, dtype: object

## The Luddite Arugment

We first show the data group by year and type of intersection or junction control.

In [1]:
df.groupby(['machine', 'Year']).describe()

NameError: name 'df' is not defined

The Luddite argument is that machine controlled intersections have a higher mean number of casulaties every year. In other words, in 2005, 1.362 is less than 1.377 and so one.

You argue that the table provides one estimate of the mean, and we'd really like to have an *interval estimate* -- it may be that some values were not reported, or there was mis-reporting or that values will differ in others years.

One way to compare this is to conduct a *hypothesis test*, and we will do that later in the semester. For now, we're going to compare our interval estimates for the mean with machines and without and if they overlap, we'll agree that the machine factor has no impact.

## Examine accident casualties

We are going to compare the number of accident casulaties in our data from machine controlled and non-machine controlled intersections. We'll first examine the data for the non-machine controlled intersections.

Create a vector `data_nonm` of the number of casualties for non-machine controlled intersections.

In [ ]:
# your code here

non_machine_controlled = df[df['machine'] == 'other']
data_nonm = non_machine_controlled['Number_of_Casualties']
data_nonm

Print the mean of the casulaties at non-machine intersections.

In [ ]:
np.mean(data_nonm)

Prepare a similar set of data for machine controlled intersections called `data_m`.

In [ ]:
# your code here
machine_controlled = df[df['machine'] == 'Automatic traffic signal']
data_m = machine_controlled['Number_of_Casualties']

Print out the mean of that data.

In [ ]:
np.mean(data_m)

Print the difference between in the means of`data_m` and `data_nonm`. It should be small but positive.

In [ ]:
sigma = np.mean(data_m) - np.mean(data_nonm)
sigma

## A Reality Check

The two data sets have very similar means, and the value of the means should indicate that the Luddites have a point. This rocks you to your machine-loving core and you decide to make certain reality is still in your grasp.

You decide to first determine that the data for the non-machine casualties is not some how violating Markov's Inequality. That is, you seek to prove:

In [ ]:
print('P(X > 2*', np.mean(data_nonm), '| < 1/2 )')

Print out the relationship with in the form $P(X > t E[X]) = l $ for $t=2$ and substituting $E[X]$ and $l$ with the values from the data. Then print out True or False if the Markov inequality would be met using an equation involving the values for  $l$ and $t$. In other words, you evaluate an expression involving `data_nonm` and $t$ that yields True or False. You may use intermediate computations if that simplifies your code.

In [ ]:
# your code here
exp_mean = 2*np.mean(data_nonm)

a = 0
b = len(data_nonm)
for row in data_nonm:
    if row > exp_mean:
        a += 1
c = a /(b)

if c < .5:
    print('The Markov inequality would stand true\n')
    print(c) 
else:
    print('False')
    print(c)

## Perhaps it's a fluke

There's a chance that the difference in the means is a fluke and that the varability of the data changes the conclusion that the machines are killing people. 

You decide to use Chebeyshev's inequality to determine if $P(|X-E[X]| > \delta) \leq 0.05%$ where $\delta$ is the difference between the mean of `data_m` and `data_nonm_`. In other words, you're trying to determine $P(|X-E[X]| < \delta) \geq 0.95$ for the `data_m` data -- that 95% of the `data_m` samples are within $\delta$ of the sample mean. If that's true, then you believe (perhaps incorrectly) that there's strong evidence that the means are "statistically identical".

Print out the expression for Chebyshev's inequality, $P(|X-E[X]| < \delta) = p$, substituting $\delta$ for the difference of the means and $p$ with the actual probability that the differences are less than $\delta$. 

Your output should be formated like `P(|X-E[X]|] <  1.2345 ) = 1.2345` but using the proper bound and actual percentage of the values for which the bound is true.

Then, print True or False if Chebyshev's inequality holds given the sample mean and variance, $\delta$ and $p$. In other words, you shoudl evaluate a conditional expression involving the $p$ computed above, the variance of the data and $\delta$.

In [ ]:
# your code here
expected_mean_m = np.mean(data_m)
sigma = np.mean(data_m) - np.mean(data_nonm)


a = 0
b = len(data_m)
for row in data_m:
    if abs(row - expected_mean_m) < sigma:
        a+=1
        
        
prob = a/b

print('P(|X-E[X]| < ', sigma, ') = ', prob, '\n')

if prob < .05:
    print("True")
else:
    print("False")


## Perhaps it's normal to be doubtful?

Things are looking bad for the machine lovers; to date, there's every reason to believe that machines at intersections cause more casualities thatn non-machine methods. The means are similar, but machines have slightly more casulaties. It's not likely that the difference between machines and non-machine casualties is so small that it's insigificant according to Chebyshev's inequality. 

You recognize that using *confidence intervals* may provide a solution. If the confidence intervals don't overlap, it's clear the means are different; if they do, you're willing to believe they're similar (again, perhaps incorrectly).

You recall that the Central Limit Theorem says that as $n \rightarrow \infty$, $\sqrt{n}\frac{\bar X_n - \mu}{\sigma}$ has a standard normal distribution. This leaves you confused, because when you look at distribution of data using a quantile-quantile plot, the number of casulaties is decidely **not** normal.

In [ ]:
stats.probplot(data_m,plot=plt);

You consult your guru who points out you've misunderstood the CLT. The CLT says that *samples of the estimate of mean* will be normally distributed.

Construct `data_m_ests` as 1000 samples of the mean of the `data_m` data using the same procedure we used in class for *bootstrapping* your data. This will involve the `np.random.choice` function. Each estimate of the mean should involve at least 1000 samples.

In [ ]:
data_m_ests = clt(data_m,1000,1000)


In [ ]:
# your code here


data_m_ests = np.random.choice(data_m,1000)

# print(np.mean(data_m_ests))

# print(np.mean(data_m))
# print(data_m_ests)
# data_m_ests = clt(data_m,1000,1000)

We will plot the histogram and the QQplot of the `data_m_ests` data:

In [ ]:
plt.hist(data_m_ests);

In [ ]:
stats.probplot(data_m_ests,plot=plt);

## Compute the confidence interval using the Student-T distribution

You recall that it's usually safer to use the Student-T distribution if the underlying dataset is not normal. 

Using the Student-T distribution, compute the 95% confidence interval for both `data_m` and `data_nonm`. Print out the lower (2.5%), the mean and the upper (97.5%) limits. 

In [ ]:
#your code here
means = ( [ np.mean( np.random.choice(data_m, data_m.size) ) for x in range(50) ] )
data_m_range = np.percentile(means, [2.5, 97.5])
print('data_m has mean', np.mean(data_m), 'with lower CI', data_m_range[0], 'and upper CI', data_m_range[1])

In [ ]:
# your code here
nonm_means = ( [ np.mean( np.random.choice(data_nonm, data_nonm.size) ) for x in range(50) ] )
data_nonm_range = np.percentile(nonm_means, [2.5, 97.5])
print('data_nonm has mean', np.mean(data_nonm), 'with lower CI', data_nonm_range[0], 'and upper CI', data_nonm_range[1])

Do the confidence intervals overlap? I.e. is there an overlap in the range of estimates of $\bar x_n$?

Yes the confidence intervals and estimates of Xn overlap. They are almost identical. 

## Compute the Confidence Interval Using Large Sample Assumption

You suddenly realize that you have have a lot of data and it's likely that the large data assumption can hold and you can use the standard normal rather than the Student-T distribution. You decide to do that to see if it changes the outcome.

Using the standard normal distribution, compute the 95% confidence interval for both `data_m` and `data_nonm`. Print out the lower (2.5%), the mean and the upper (97.5%) limits. 

In [ ]:
Xbar = np.array([np.mean(np.random.choice(data_m, 1000)) for x in range(1000)])
mu=np.mean(data_m)
sigma = Xbar.std()

z = (Xbar - mu)/sigma
z = np.percentile(Xbar, [2.5, 97.5])
print('data_m has mean', np.mean(data_m), 'with lower CI', z[0], 'and upper CI', z[1])

In [ ]:
# your code here
nonm_Xbar = np.array([np.mean(np.random.choice(data_nonm, 1000)) for x in range(200000)])
nonm_mu=np.mean(data_nonm)
nonm_sigma = nonm_Xbar.std()

nonm_z = (nonm_Xbar - nonm_mu)/nonm_sigma
nonm_z = np.percentile(nonm_Xbar, [2.5, 97.5])
print('data_m has mean', np.mean(data_nonm), 'with lower CI', nonm_z[0], 'and upper CI', nonm_z[1])

In [ ]:
# your code her 

Does this change the results of your comparisons of the mean?

YOUR ANSWER HERE

## Bootstrap Confidence Intervals

Just to leave no stone un-turned, you decide to determine bootstrap confidence intervals to see if they're consistent with the prior confidence intervals.

To do this, it will be easiest to write a function `tstar(x,m)` that computes the bootstrap `t*` value for sample `x` and overall mean `m`. By this time in the class, we're going to assume you have learned how to write a simple function.

Provide function `t`:

In [ ]:
def tstar(xi, mean):
    se = np.std(xi, ddof=1) / np.sqrt(len(xi))
    return np.mean(xi - mean) / se

Using the bootstrap confidence interval process, compute the 95% confidence interval for both `data_m` and `data_nonm`. Print out the lower (2.5%), the mean and the upper (97.5%) limits. You should also print out the lower and upper estimates $c_l$ and $c_u$ determined by the bootstrap. Use 1000 sample means of 20000 values.

In [ ]:
# your code here
#code here
data_m
mu = np.mean(data_m)

def tstar(xi, mean):
    se = np.std(xi, ddof=1) / np.sqrt(len(xi))
    return np.mean(xi - mean) / se

t_vals = np.array([tstar(np.random.choice(data_m,10000),mu) for x in range(200000)])
c1,c2=np.percentile(t_vals, [2.5, 97.5])
print("c1 is :", c1, "Mean is: ", mu, "c2 is:",c2)

In [ ]:
# your code here
data_nonm
dmu = np.mean(data_nonm)

def tstar(xi, mean):
    se = np.std(xi, ddof=1) / np.sqrt(len(xi))
    return np.mean(xi - mean) / se

d_vals = np.array([tstar(np.random.choice(data_nonm,1000),dmu) for x in range(200000)])
d_c1,d_c2=np.percentile(d_vals, [2.5, 97.5])
print("c1 is :", d_c1, "Mean is: ", dmu, "c2 is:",d_c2)

Are the confidence intervals symmetric? I.e. is $c_l = -c_u$? Also, do your earlier conclusions about the means still hold? Since the bootstrap depends on random samples, you may want to run this a few times befor answering.

The Confidence intervals are semetric and because the mean is within the two values, the mean still holds. These were the only two intervals that dont match the others, so there might be an error in my calculations.

## Can I just bootstrap the data?

Since you have a fair amount of data, you decide to settle the issue using an *empirical bootstrap* to estimate the distribution of the mean (*i.e.* not a bootstrap confidence interval).

For both `data_m` and `data_nonm`, compute the 95% confidence interval using a boostrap with 20,000 samples repeated 1,000 times.  For each dataset, print out the confidence interval in terms of (lower_bound, mean, upper_bound).

In [ ]:
# your code here
def boot(x):
    mean = np.mean(x)
    boot_means = ([np.mean(np.random.choice(x,1000))for i in range(200000)])
    c1,c2=np.percentile(boot_means, [2.5, 97.5])
    return c1,mean,c2
boot(data_m)

In [ ]:
# your code here
def boot(x):
    mean = np.mean(x)
    boot_means = ([np.mean(np.random.choice(x,1000))for i in range(200000)])
    c1,c2=np.percentile(boot_means, [2.5, 97.5])
    return c1,mean,c2
boot(data_nonm)

Do the conclusions change compared to the previous methods?

Yes, these values were slightly different to the bootstrap method from before, which makes me think that my bootstrap formula was probably off.

## Comparison

In the cell below, you'll find a table into which you should enter your lower mean estimate, mean and upper mean estimate for each of the indicated techniques.

Round each result to four siginificant digits.

| Method | Dataset | $c_l$ | $\bar x_n$ | $c_u$ |
| :---:  | :------:|:-----:|:----------:|:----:|
| T-dist | m | 1.3663 | 1.3693  | 1.3756  | 
| T-dist | nonm | 1.3604 | 1.3628  | 1.36435 |
| Normal | M | 1.3179  | 1.3693 | 1.424 |
| Normal | nonm | 1.314 | 1.3628 | 1.4130|
| Boot CI | m | -1.9017 | 1.3693  | 1.856 |
| Boot CI | nonm | -2.0977 | 1.3628 | 1.9578 |
| Emp. Boot | m | 1.321 | 1.3683 | 1.423 |
| Emp. Boot | nonm | 1.313 | 1.362 | 1.416 |

YOUR ANSWER HERE


Examine this table and comment on the relationship between the confidence intervals from the T-distribution and the Normal distribution. Are they similar? Also comment on the relationship between the Normal distribution and the Bootstrap CI. Lastly, comment on the Bootstrap CI and the Empirical Bootstrap estimate; before answering the question, go back to your Empirical Bootstrap and draw much larger samples (e.g. 1000 samples of 200,000 rather than 20,000). 

The T-distribution and the Normal Distribution are very similar in thier results but my bootstrap CI was off compared to the other calculations. the normal distribution compared to the emperical bootstrap was relatively close to eachother. Almost identical. 

# Problem #2 - Speed of light [25 points]

We will now address a smaller problem where we'll see more stark differences between the techniques.

We're going to use a reduced version of the the Michelson speed of light data. Because the Michelson data has 100 elements, we would find that the Student-T and Standard Normal distibution would give almost identical results. Although that's not "big data", it's useful to understand the difference between the Student-T and normal distribution in practice.

In [ ]:
spd = np.array([  850.,   740.,   900.,  1070.,   930.,   850.,   950.,   980.,
         980.,   880.,  1000.,   980.,   930.,   650.,   760.,   810.,
        1000.,  1000.,   960.,   960.,   960.,   940.,   960.,   940.,
         880.,   800.,   850.,   880.,   900.,   840.,   830.,   790.,
         810.,   880.,   880.,   830.,   800.,   790.,   760.,   800.,
         880.,   880.,   880.,   860.,   720.,   720.,   620.,   860.,
         970.,   950.,   880.,   910.,   850.,   870.,   840.,   840.,
         850.,   840.,   840.,   840.,   890.,   810.,   810.,   820.,
         800.,   770.,   760.,   740.,   750.,   760.,   910.,   920.,
         890.,   860.,   880.,   720.,   840.,   850.,   850.,   780.,
         890.,   840.,   780.,   810.,   760.,   810.,   790.,   810.,
         820.,   850.,   870.,   870.,   810.,   740.,   810.,   940.,
         950.,   800.,   810.,   870.]) + 299000

This data is approximately normal because the deviation of the values represent measurement error in a constrained environment while measuring the speed of light.

We're going to select a subset of data values to be dataset. You'll be running this code repeatedly to answer a question below after changing the size of the dataset. I've prepared a random vector of locations using `np.random.choice` that we'll use to extract the data:

In [ ]:
indicies = np.random.choice(range(100), 5)

In [ ]:
speed = pd.Series( spd[indicies] )

Now, calculate `speed_x` where x is `m`, `v`, `n` and `se` representing the mean, variance, sample size and *standard error*. The standard error is the quantity $\sqrt{Var[X]/n} = \sigma/\sqrt{n}$, which is used computing confidence intervals using either the T-distribution or the standard normal approximation.

In [ ]:
speed_m = np.mean(speed)
speed_v = np.var(speed)
speed_n = len(speed)
speed_se = np.sqrt(speed_v / speed_n)

Compute a 3-tuple or 3-list named `t_ci` that contains the lower confidence limit, mean and upper confidence limit using a Student-T distribution for a 95% confidence interval.

In [ ]:
# your code here


Do the same for a variable named `n_ci` for the confidence interval using the standard normal assumption. Although the data is approximately normally distributed, the standard normal assumption is questionable because the sample size is small (10) and the data deviates a little from normal.

In [ ]:
# your code here


Now, draw a kernel density estimate (KDE) plot of the data; plot the mean using a verticle redline (using `plt.axvline`) and then the Student-T confidence interval in blue and the standard normal confidence interval in green.

In [ ]:
# your code here


Run the code repeatedly starting with 10 samples until you find the smallest value where you can no longer distinguish the difference between the Student-T distribution and the standard normal from the plot. Enter that number below.

YOUR ANSWER HERE

# Confidence limits of polling data [25 points]

*N.B.* Although this problem involves political polling data, I want to assure you that we're just looking for a good dataset for polling.

The website [538](https://projects.fivethirtyeight.com/trump-approval-ratings/?ex_cid=rrpromo) runs a series of "weighted" polls to estimate e.g. the popularity of the current president or other electorial outcomes.

They include a number of polls. One of them, the [Feb 25-March 1 2018 IPSO poll](http://polling.reuters.com/#poll/CP3_2/), shows an approval rating of 38.2%, with a reported range of 35.6%-40.9% with a sample size of 1488. This means that $n*\hat p = 568$ people of 1488 reported approval.

There's no mention of how the estimate range 35.6%-40.9% is determined. Let's see if we can reverse engineeer the different methods we've heard about and see which they use.

In [ ]:
n = 1488
support = 568
pest = support/n

### Wilson estimate

Using the "Wilson method" described in the text, report the 95% confidence interval. You can either work out the solution long hand and then enter the calculations below to determine the confidence interval or you [can cheat and use the SymPy package to solve the equations](http://docs.sympy.org/latest/modules/solvers/solvers.html).

In [ ]:
# your code here


### Normal approximation with specified $\hat p$

The central limit theorem says that estimates for $S_n = X_1 + \ldots + X_n$ should follow a standard normal distribution with mean $E[S_n]/n = p$ and variance $Var[S_n] = Var[X]/n$ or $\sigma = \frac{\sqrt{p(1-p)}}{n}$. 

Perhaps Reuter's uses the specified value of $\hat p$ in order to determine the variance or standard deviation? Run the numbers to compute the 95% confidence interval using the standard normal approximation.

In [ ]:
# your code here


### Student-T approximation with specified $\hat p$

Perhaps they're using the Student-T distribution instead of the standard normal despite the large sample size? Give that a try and compute the 95% confidence interval using the student-T distribution.

In [ ]:
# your code here


### Worst case variance?

As mentioned in lecture, if we don't know the precise value of $\hat p$, it's difficult to estimate the varince since that depends on $\hat p$; the Wilson method is precise but annoyingly complex.

However, because we know that the variance is maximal at $p=1/2$ we can use that to compute a conservative bound. Use that approximation and the standard normal assumption to compute a bound on the 95% confidence interval.

In [ ]:
# your code here


### Give it your best guess.

What method do you think Reuter's used?

YOUR ANSWER HERE

In [ ]:
#t-test data_nonm

data_nonm
dmu = np.mean(data_nonm)

def tsample(xi, mean):
    se = np.std(xi, ddof=1) / np.sqrt(len(xi))
    return np.mean(xi - mean) / se

d_vals = np.array([tsample(np.random.choice(data_nonm,50),dmu) for x in range(1000)])
c1,c2=np.percentile(d_vals, [2.5, 97.5])
print("c1 is :", c1, "Mean is: ", mu, "c2 is:",c2)

In [ ]:
#t-test data_m

#code here
data_m
mu = np.mean(data_m)

def tsample(xi, mean):
    se = np.std(xi, ddof=1) / np.sqrt(len(xi))
    return np.mean(xi - mean) / se

t_vals = np.array([tsample(np.random.choice(data_m,50),mu) for x in range(1000)])
c1,c2=np.percentile(t_vals, [2.5, 97.5])
print("c1 is :", c1, "Mean is: ", mu, "c2 is:",c2)